In [1]:
# ----------------------------------------------------------------------
# Numenta Platform for Intelligent Computing (NuPIC)
# Copyright (C) 2019, Numenta, Inc.  Unless you have an agreement
# with Numenta, Inc., for a separate license for this software code, the
# following terms and conditions apply:
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero Public License version 3 as
# published by the Free Software Foundation.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Affero Public License for more details.
#
# You should have received a copy of the GNU Affero Public License
# along with this program.  If not, see http://www.gnu.org/licenses.
#
# http://numenta.org/licenses/
# ----------------------------------------------------------------------

In [109]:
!pip install git+https://github.com/numenta/nupic.torch.git#egg=nupic.torch
!pip install torch torchvision

  Cloning https://github.com/numenta/nupic.torch.git to /tmp/pip-install-4az9ys1o/nupic.torch
  Running command git clone -q https://github.com/numenta/nupic.torch.git /tmp/pip-install-4az9ys1o/nupic.torch
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 748.8MB 20kB/s 
  Created wheel for nupic.torch: filename=nupic.torch-0.0.1.dev0-cp37-none-any.whl size=31974 sha256=3618b445f6362b752ff1faef21440deff670d41f16fa62599705c153bae0d7b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-9blzzbe5/wheels/16/4f/f8/89dd360bad620bcbb6b468e0243812cdfeb76792b4a79b263d
Successfully built nupic.torch
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.6.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.6.0 which is incompatible.
  Found existing installation:

In [2]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm_notebook as tqdm

SEED = 18
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [107]:
def train(model, loader, optimizer, criterion, post_batch_callback=None):
    """
    Train the model using given dataset loader. 
    Called on every epoch.
    :param model: pytorch model to be trained
    :type model: torch.nn.Module
    :param loader: dataloader configured for the epoch.
    :type loader: :class:`torch.utils.data.DataLoader`
    :param optimizer: Optimizer object used to train the model.
    :type optimizer: :class:`torch.optim.Optimizer`
    :param criterion: loss function to use
    :type criterion: function
    :param post_batch_callback: function(model) to call after every batch
    :type post_batch_callback: function
    """
    model.train()
    for batch_idx, (data, target) in enumerate(tqdm(loader, leave=False)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if post_batch_callback is not None:
            post_batch_callback(model)


def test(model, loader, criterion):
    """
    Evaluate pre-trained model using given dataset loader.
    Called on every epoch.
    :param model: Pretrained pytorch model
    :type model: torch.nn.Module
    :param loader: dataloader configured for the epoch.
    :type loader: :class:`torch.utils.data.DataLoader`
    :param criterion: loss function to use
    :type criterion: function
    :return: Dict with "accuracy", "loss" and "total_correct"
    """
    model.eval()
    loss = 0
    total_correct = 0
    with torch.no_grad():
        for data, target in tqdm(loader, leave=False):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            total_correct += pred.eq(target.view_as(pred)).sum().item()
    
    return {"accuracy": total_correct / len(loader.dataset), 
            "loss": loss / len(loader.dataset), 
            "total_correct": total_correct}

### Parameters

In [3]:
# Training parameters
LEARNING_RATE = 0.02
LEARNING_RATE_GAMMA = 0.8
MOMENTUM = 0.0
EPOCHS = 15
FIRST_EPOCH_BATCH_SIZE = 4
TRAIN_BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000

### Create Sparse MNIST model

There are 2 ways to create **nupic.torch** sparse models. You can import the models from **nupic.torch.models** or use pytorch's [torch.hub](https://pytorch.org/docs/stable/hub.html) API.

In this example we will import the models. 

In [130]:
#  Numenta Platform for Intelligent Computing (NuPIC)
#  Copyright (C) 2019, Numenta, Inc.  Unless you have an agreement
#  with Numenta, Inc., for a separate license for this software code, the
#  following terms and conditions apply:
#
#  This program is free software: you can redistribute it and/or modify
#  it under the terms of the GNU Affero Public License version 3 as
#  published by the Free Software Foundation.
#
#  This program is distributed in the hope that it will be useful,
#  but WITHOUT ANY WARRANTY; without even the implied warranty of
#  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
#  See the GNU Affero Public License for more details.
#
#  You should have received a copy of the GNU Affero Public License
#  along with this program.  If not, see http://www.gnu.org/licenses.
#
#  http://numenta.org/licenses/
#
import warnings
from collections import OrderedDict

from torch import nn
from torch.hub import load_state_dict_from_url

from nupic.torch.modules import (
    Flatten,
    KWinners,
    KWinners2d,
    SparseWeights,
    SparseWeights2d,
)


class SparseCNN(nn.Sequential):
    """Sparse CNN model used to classify `MNIST` dataset as described in `How
    Can We Be So Dense?`_ paper.
    .. _`How Can We Be So Dense?`: https://arxiv.org/abs/1903.11257
    :param cnn_out_channels: output channels for each CNN layer
    :param cnn_percent_on: Percent of units allowed to remain on each convolution
                           layer
    :param linear_units: Number of units in the linear layer
    :param linear_percent_on: Percent of units allowed to remain on the linear
                              layer
    :param k_inference_factor: During inference (training=False) we increase
                               `percent_on` in all sparse layers by this factor
    :param boost_strength: boost strength (0.0 implies no boosting)
    :param boost_strength_factor: Boost strength factor to use [0..1]
    :param duty_cycle_period: The period used to calculate duty cycles
    :param kwinner_local: Whether or not to choose the k-winners locally (across
                          the channels at each location) or globally (across the
                          whole input and across all channels)
    :param cnn_sparsity: Percent of weights that are zero
    :param linear_sparsity: Percent of weights that are zero.
    """

    def __init__(
        self,
        cnn_out_channels=(32, 64),
        cnn_percent_on=(0.1, 0.2),
        cnn_weight_sparsity=None,
        linear_units=700,
        linear_percent_on=0.2,
        linear_weight_sparsity=None,
        boost_strength=1.5,
        boost_strength_factor=0.85,
        k_inference_factor=1.0,
        duty_cycle_period=1000,
        kwinner_local=False,
        cnn_sparsity=(0.4, 0.55),
        linear_sparsity=0.8,
    ):
        if cnn_weight_sparsity is not None:
            warnings.warn(
                "Parameter `cnn_weight_sparsity` is deprecated. Use "
                "`cnn_sparsity` instead.",
                DeprecationWarning,
            )
            cnn_sparsity = (1.0 - cnn_weight_sparsity[0], 1.0 - cnn_weight_sparsity[1])

        if linear_weight_sparsity is not None:
            warnings.warn(
                "Parameter `linear_weight_sparsity` is deprecated. Use "
                "`linear_sparsity` instead.",
                DeprecationWarning,
            )
            linear_sparsity = 1.0 - linear_weight_sparsity

        super(SparseCNN, self).__init__(
            OrderedDict(
                [
                    # First Sparse CNN layer
                    (
                        "cnn1",
                        SparseWeights2d(
                            nn.Conv2d(1, cnn_out_channels[0], 5),
                            sparsity=cnn_sparsity[0],
                        ),
                    ),
                    ("cnn1_maxpool", nn.MaxPool2d(2)),
                    (
                        "cnn1_kwinner",
                        KWinners2d(
                            channels=cnn_out_channels[0],
                            percent_on=cnn_percent_on[0],
                            k_inference_factor=k_inference_factor,
                            boost_strength=boost_strength,
                            boost_strength_factor=boost_strength_factor,
                            duty_cycle_period=duty_cycle_period,
                            local=kwinner_local,
                        ),
                    ),
                    # Second Sparse CNN layer
                    (
                        "cnn2",
                        SparseWeights2d(
                            nn.Conv2d(cnn_out_channels[0], cnn_out_channels[1], 5),
                            sparsity=cnn_sparsity[1],
                        ),
                    ),
                    ("cnn2_maxpool", nn.MaxPool2d(2)),
                    (
                        "cnn2_kwinner",
                        KWinners2d(
                            channels=cnn_out_channels[1],
                            percent_on=cnn_percent_on[1],
                            k_inference_factor=k_inference_factor,
                            boost_strength=boost_strength,
                            boost_strength_factor=boost_strength_factor,
                            duty_cycle_period=duty_cycle_period,
                            local=kwinner_local,
                        ),
                    ),
                    ("flatten", Flatten()),
                    # Sparse Linear layer
                    (
                        "linear",
                        SparseWeights(
                            nn.Linear(47 *47 * cnn_out_channels[1], linear_units),
                            sparsity=linear_sparsity,
                        ),
                    ),
                    (
                        "linear_kwinner",
                        KWinners(
                            n=linear_units,
                            percent_on=linear_percent_on,
                            k_inference_factor=k_inference_factor,
                            boost_strength=boost_strength,
                            boost_strength_factor=boost_strength_factor,
                            duty_cycle_period=duty_cycle_period,
                        ),
                    ),
                    # Classifier
                    ("output", nn.Linear(linear_units, 10)),
                    ("softmax", nn.LogSoftmax(dim=1)),
                ]
            )
        )


class GSCSparseCNN(nn.Sequential):
    """Sparse CNN model used to classify `Google Speech Commands` dataset as
    described in `How Can We Be So Dense?`_ paper.
    .. _`How Can We Be So Dense?`: https://arxiv.org/abs/1903.11257
    :param cnn_out_channels: output channels for each CNN layer
    :param cnn_percent_on: Percent of units allowed to remain on each convolution
                           layer
    :param linear_units: Number of units in the linear layer
    :param linear_percent_on: Percent of units allowed to remain on the linear
                              layer
    :param k_inference_factor: During inference (training=False) we increase
                               `percent_on` in all sparse layers by this factor
    :param boost_strength: boost strength (0.0 implies no boosting)
    :param boost_strength_factor: Boost strength factor to use [0..1]
    :param duty_cycle_period: The period used to calculate duty cycles
    :param kwinner_local: Whether or not to choose the k-winners locally (across
                          the channels at each location) or globally (across the
                          whole input and across all channels)
    :param cnn_sparsity: Percent of weights that zero
    :param linear_sparsity: Percent of weights that are zero in the
                            linear layer.
    """

    def __init__(
        self,
        cnn_out_channels=(64, 64),
        cnn_percent_on=(0.095, 0.125),
        cnn_weight_sparsity=None,
        linear_units=1000,
        linear_percent_on=0.1,
        linear_weight_sparsity=None,
        boost_strength=1.5,
        boost_strength_factor=0.9,
        k_inference_factor=1.0,
        duty_cycle_period=1000,
        kwinner_local=False,
        cnn_sparsity=(0.5, 0.8),
        linear_sparsity=0.9,
    ):
        super(GSCSparseCNN, self).__init__()

        if cnn_weight_sparsity is not None:
            warnings.warn(
                "Parameter `cnn_weight_sparsity` is deprecated. Use "
                "`cnn_sparsity` instead.",
                DeprecationWarning,
            )
            cnn_sparsity = (1.0 - cnn_weight_sparsity[0], 1.0 - cnn_weight_sparsity[1])

        if linear_weight_sparsity is not None:
            warnings.warn(
                "Parameter `linear_weight_sparsity` is deprecated. Use "
                "`linear_sparsity` instead.",
                DeprecationWarning,
            )
            linear_sparsity = 1.0 - linear_weight_sparsity

        # input_shape = (1, 32, 32)
        # First Sparse CNN layer
        if cnn_sparsity[0] > 0:
            self.add_module(
                "cnn1",
                SparseWeights2d(
                    nn.Conv2d(1, cnn_out_channels[0], 5), sparsity=cnn_sparsity[0]
                ),
            )
        else:
            self.add_module("cnn1", nn.Conv2d(1, cnn_out_channels[0], 5))
        self.add_module(
            "cnn1_batchnorm", nn.BatchNorm2d(cnn_out_channels[0], affine=False)
        )
        self.add_module(
            "cnn1_kwinner",
            KWinners2d(
                channels=cnn_out_channels[0],
                percent_on=cnn_percent_on[0],
                k_inference_factor=k_inference_factor,
                boost_strength=boost_strength,
                boost_strength_factor=boost_strength_factor,
                duty_cycle_period=duty_cycle_period,
                local=kwinner_local,
            ),
        )
        self.add_module("cnn1_maxpool", nn.MaxPool2d(2))

        # Second Sparse CNN layer
        if cnn_sparsity[1] > 0:
            self.add_module(
                "cnn2",
                SparseWeights2d(
                    nn.Conv2d(cnn_out_channels[0], cnn_out_channels[1], 5),
                    sparsity=cnn_sparsity[1],
                ),
            )
        else:
            self.add_module(
                "cnn2", nn.Conv2d(cnn_out_channels[0], cnn_out_channels[1], 5)
            )
        self.add_module(
            "cnn2_batchnorm", nn.BatchNorm2d(cnn_out_channels[1], affine=False)
        )
        self.add_module(
            "cnn2_kwinner",
            KWinners2d(
                channels=cnn_out_channels[1],
                percent_on=cnn_percent_on[1],
                k_inference_factor=k_inference_factor,
                boost_strength=boost_strength,
                boost_strength_factor=boost_strength_factor,
                duty_cycle_period=duty_cycle_period,
                local=kwinner_local,
            ),
        )
        self.add_module("cnn2_maxpool", nn.MaxPool2d(2))

        self.add_module("flatten", Flatten())

        # Sparse Linear layer
        self.add_module(
            "linear",
            SparseWeights(
                nn.Linear(25 * cnn_out_channels[1], linear_units),
                sparsity=linear_sparsity,
            ),
        )
        self.add_module("linear_bn", nn.BatchNorm1d(linear_units, affine=False))
        self.add_module(
            "linear_kwinner",
            KWinners(
                n=linear_units,
                percent_on=linear_percent_on,
                k_inference_factor=k_inference_factor,
                boost_strength=boost_strength,
                boost_strength_factor=boost_strength_factor,
                duty_cycle_period=duty_cycle_period,
            ),
        )

        # Classifier
        self.add_module("output", nn.Linear(linear_units, 12))
        self.add_module("softmax", nn.LogSoftmax(dim=1))


class GSCSuperSparseCNN(GSCSparseCNN):
    """Super Sparse CNN model used to classify `Google Speech Commands`
    dataset as described in `How Can We Be So Dense?`_ paper.
    This model provides a sparser version of :class:`GSCSparseCNN`
    .. _`How Can We Be So Dense?`: https://arxiv.org/abs/1903.11257
    """

    def __init__(self):
        super(GSCSuperSparseCNN, self).__init__(linear_units=1500, linear_sparsity=0.95)


MODEL_URLS = {
    "gsc_sparse_cnn": "http://public.numenta.com/pytorch/hub/gsc_sparse_cnn-7bc3782d.pth",  # noqa: E501
    "gsc_super_sparse_cnn": "http://public.numenta.com/pytorch/hub/gsc_super_sparse_cnn-d412de15.pth",  # noqa: E501
}


def gsc_sparse_cnn(pretrained=False, progress=True, **kwargs):
    """
    Sparse CNN model used to classify 'Google Speech Commands' dataset
    :param pretrained: If True, returns a model pre-trained on Google Speech Commands
    :param progress: If True, displays a progress bar of the download to stderr
    :param kwargs: See :class:`GSCSparseCNN`
    """
    model = GSCSparseCNN(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(
            MODEL_URLS["gsc_sparse_cnn"], progress=progress
        )
        model.load_state_dict(state_dict)
    return model


def gsc_super_sparse_cnn(pretrained=False, progress=True):
    """
    Super Sparse CNN model used to classify `Google Speech Commands`
    dataset as described in `How Can We Be So Dense?`_ paper.
    This model provides a sparser version of :class:`GSCSparseCNN`
    :param pretrained: If True, returns a model pre-trained on Google Speech Commands
    :param progress: If True, displays a progress bar of the download to stderr
    """
    model = GSCSuperSparseCNN()
    if pretrained:
        state_dict = load_state_dict_from_url(
            MODEL_URLS["gsc_super_sparse_cnn"], progress=progress
        )
        model.load_state_dict(state_dict)
    return model

In [131]:

# For this example we will use the default values. 
# See MNISTSparseCNN documentation for all possible parameters and their values.
model = SparseCNN().to(device)
print(model)

SparseCNN(
  (cnn1): SparseWeights2d(
    sparsity=0.4
    (module): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  )
  (cnn1_maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn1_kwinner): KWinners2d(channels=32, local=False, break_ties=False, n=0, percent_on=0.1, boost_strength=1.5, boost_strength_factor=0.85, k_inference_factor=1.0, duty_cycle_period=1000)
  (cnn2): SparseWeights2d(
    sparsity=0.55
    (module): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  )
  (cnn2_maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2_kwinner): KWinners2d(channels=64, local=False, break_ties=False, n=0, percent_on=0.2, boost_strength=1.5, boost_strength_factor=0.85, k_inference_factor=1.0, duty_cycle_period=1000)
  (flatten): Flatten()
  (linear): SparseWeights(
    sparsity=0.8
    (module): Linear(in_features=141376, out_features=700, bias=True)
  )
  (linear_kwinner): KWinners(n=700, percent_on=0.2, boost_

### Load MNIST Dataset

In [99]:
normalize = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('data', train=True, download=True, transform=normalize)
test_dataset = datasets.MNIST('data', train=False, transform=normalize)

# Configure data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=True)
first_loader = torch.utils.data.DataLoader(train_dataset, batch_size=FIRST_EPOCH_BATCH_SIZE, shuffle=True)


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [115]:
train_loader.dataset[0][0].size()

torch.Size([1, 28, 28])

In [102]:
from scipy import io
from torch.utils.data import TensorDataset
import pandas as pd

data = []
y = []

# get subject_ID and y
dataset = pd.read_csv(r'/content/Phenotypic_V1_0b_preprocessed1.csv')
target = dict()
for i in range(len(dataset)):
  target[dataset['subject'][i]] = dataset['DX_GROUP'][i]

# get data
mat = io.loadmat('/content/50003_cc200_correlation.mat')
data_mat = mat['connectivity'] # use the key for data here
data.append(np.reshape(np.array(data_mat), (1, 200, 200)))
y.append(target[50003])

mat = io.loadmat('/content/50004_cc200_correlation.mat')
data_mat = mat['connectivity'] # use the key for data here
data.append(np.reshape(np.array(data_mat), (1, 200, 200)))
y.append(target[50004])

data = torch.from_numpy(np.array(data)).float()
target = torch.from_numpy(np.array(y)).long() # change type to your use case

dataset = TensorDataset(data, target)
dataset


In [116]:
# Configure data loaders
data_loader = torch.utils.data.DataLoader(dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
data_loader.dataset[0][0].size()

torch.Size([1, 200, 200])

### Train
On the first epoch we use smaller batch size to calculate the duty cycles used by the k-winner function. Once the duty cycles stabilize we can use larger batch sizes. Using the `post_batch`, we rezero the weights after every batch to keep the initial sparsity constant.

In [132]:
from nupic.torch.modules import rezero_weights, update_boost_strength

def post_batch(model):
    model.apply(rezero_weights)

sgd = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
lr_scheduler = optim.lr_scheduler.StepLR(sgd, step_size=1, gamma=LEARNING_RATE_GAMMA)
train(model=model, loader=data_loader, optimizer=sgd, criterion=F.nll_loss, post_batch_callback=post_batch)
lr_scheduler.step()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


After each we apply the boost strength factor

In [133]:
%%capture
model.apply(update_boost_strength)

Test and print results

In [134]:
test(model=model, loader=data_loader, criterion=F.nll_loss)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


{'accuracy': 1.0, 'loss': 2.260118007659912, 'total_correct': 2}

At this point the duty cycles should be stable and we can train on larger batch sizes

In [11]:
for epoch in range(1, EPOCHS):
    train(model=model, loader=train_loader, optimizer=sgd, criterion=F.nll_loss, post_batch_callback=post_batch)
    lr_scheduler.step()
    model.apply(rezero_weights)
    model.apply(update_boost_strength)
    results = test(model=model, loader=test_loader, criterion=F.nll_loss)
    print(results)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


{'accuracy': 0.9879, 'loss': 0.032748318862915037, 'total_correct': 9879}


{'accuracy': 0.9898, 'loss': 0.03100272102355957, 'total_correct': 9898}


{'accuracy': 0.9892, 'loss': 0.03043186492919922, 'total_correct': 9892}


...

{'accuracy': 0.9897, 'loss': 0.02871354923248291, 'total_correct': 9897}


{'accuracy': 0.9894, 'loss': 0.02861289234161377, 'total_correct': 9894}


{'accuracy': 0.9899, 'loss': 0.028770042037963867, 'total_correct': 9899}


{'accuracy': 0.9896, 'loss': 0.028808555030822754, 'total_correct': 9896}


{'accuracy': 0.9899, 'loss': 0.028980622100830078, 'total_correct': 9899}


### Noise
Add noise to the input and check the test accuracy

In [12]:
class RandomNoise(object):
    """
    An image transform that adds noise to random pixels in the image.
    """
    def __init__(self, noise_level=0.0, white_value=0.1307 + 2*0.3081):
        """
        :param noise_level:
          From 0 to 1. For each pixel, set its value to white_value with this
          probability. Suggested white_value is 'mean + 2*stdev'
        """
        self.noise_level = noise_level
        self.white_value = white_value

    def __call__(self, image):
        a = image.view(-1)
        num_noise_bits = int(a.shape[0] * self.noise_level)
        noise = np.random.permutation(a.shape[0])[0:num_noise_bits]
        a[noise] = self.white_value
        return image

In [13]:
noise_score = 0
for noise in [0.0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
    noise_transform = transforms.Compose([transforms.ToTensor(), RandomNoise(noise), 
                                      transforms.Normalize((0.1307,), (0.3081,))])
    noise_dataset = datasets.MNIST('data', train=False, transform=noise_transform)
    noise_loader = torch.utils.data.DataLoader(noise_dataset, 
                                               batch_size=TEST_BATCH_SIZE, 
                                               shuffle=True)

    results = test(model=model, loader=noise_loader, criterion=F.nll_loss)
    noise_score += results["total_correct"]
    print(noise, ":", results)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0.0 : {'accuracy': 0.9899, 'loss': 0.028980636405944823, 'total_correct': 9899}


0.05 : {'accuracy': 0.9886, 'loss': 0.032598723220825195, 'total_correct': 9886}


0.1 : {'accuracy': 0.9869, 'loss': 0.03899371376037598, 'total_correct': 9869}


0.15 : {'accuracy': 0.9831, 'loss': 0.0483180721282959, 'total_correct': 9831}


...

0.3 : {'accuracy': 0.966, 'loss': 0.1071176155090332, 'total_correct': 9660}


0.35 : {'accuracy': 0.952, 'loss': 0.14186668853759765, 'total_correct': 9520}


0.4 : {'accuracy': 0.9389, 'loss': 0.1936940658569336, 'total_correct': 9389}


0.45 : {'accuracy': 0.9097, 'loss': 0.27848018646240236, 'total_correct': 9097}


0.5 : {'accuracy': 0.8724, 'loss': 0.39100286560058595, 'total_correct': 8724}


In [14]:
print("noise_score:", noise_score)

noise_score: 105415
